# Reflection Pattern

The first pattern we are going to implement is the **reflection pattern**. 

---

<img src="../img/reflection_pattern.png" alt="Alt text" width="600"/>

---

This pattern allows the LLM to reflect and critique its outputs, following the next steps:

1. The LLM **generates** a candidate output. If you look at the diagram above, it happens inside the **"Generate"** box.
2. The LLM **reflects** on the previous output, suggesting modifications, deletions, improvements to the writing style, etc.
3. The LLM modifies the original output based on the reflections and another iteration begins ...

**Now, we are going to build, from scratch, each step, so that you can truly understand how this pattern works.**

# Generation Step

The first thing we need to consider is:

> What do we want to generate? A poem? An essay? Python code?

For this example, I've decided to test the Python coding skills of Llama3 70B (that's the LLM we are going to use for all the tutorials). In particular, we are going to ask our LLM to code a famous sorting algorithm: **Merge Sort**. 

---

<img src="../img/mergesort.png" alt="Alt text" width="500"/>

### Groq Client and relevant imports

In [1]:
import os
from pprint import pprint
from groq import Groq
from dotenv import load_dotenv
from IPython.display import display_markdown

# Remember to load the environment variables. You should have the Groq API Key in there :)
load_dotenv()

client = Groq()

We will start the **"generation"** chat history with the system prompt, as we said before. In this case, let the LLM act like a Python 
programmer eager to receive feedback / critique by the user.

In [2]:
generation_chat_history = [
    {
        "role": "system",
        "content": "You are a Python programmer tasked with generating high quality Python code."
        "Your task is to Generate the best content possible for the user's request. If the user provides critique," 
        "respond with a revised version of your previous attempt."
    }
]

Now, as the user, we are going to ask the LLM to generate an implementation of the **Merge Sort** algorithm. Just add a new message with the **user** role to the chat history.

In [3]:
generation_chat_history.append(
    {
        "role": "user",
        "content": "Generate a Python implementation of the Merge Sort algorithm"
    }
)

Let's generate the first version of the essay.

In [4]:
mergesort_code = client.chat.completions.create(
    messages=generation_chat_history,
    model="llama3-70b-8192"
).choices[0].message.content

generation_chat_history.append(
    {
        "role": "assistant",
        "content": mergesort_code
    }
)

In [5]:
display_markdown(mergesort_code, raw=True)

Here is a Python implementation of the Merge Sort algorithm:

```python
def merge_sort(arr):
    """
    Sorts an array using the Merge Sort algorithm.

    :param arr: The array to be sorted
    :return: A new sorted array
    """
    # Base case: If the array has only one element, it is already sorted
    if len(arr) <= 1:
        return arr

    # Find the middle of the array
    mid = len(arr) // 2

    # Divide the array into two halves
    left_half = arr[:mid]
    right_half = arr[mid:]

    # Recursively sort the two halves
    left_half = merge_sort(left_half)
    right_half = merge_sort(right_half)

    # Merge the two sorted halves
    return merge(left_half, right_half)


def merge(left, right):
    """
    Merges two sorted arrays into one sorted array.

    :param left: The first sorted array
    :param right: The second sorted array
    :return: A new sorted array
    """
    merged = []
    left_index = 0
    right_index = 0

    # Merge the two arrays
    while left_index < len(left) and right_index < len(right):
        if left[left_index] < right[right_index]:
            merged.append(left[left_index])
            left_index += 1
        else:
            merged.append(right[right_index])
            right_index += 1

    # Add any remaining elements from the left and right arrays
    merged.extend(left[left_index:])
    merged.extend(right[right_index:])

    return merged


# Example usage
arr = [64, 34, 25, 12, 22, 11, 90]
sorted_arr = merge_sort(arr)
print(sorted_arr)
```

This implementation defines two functions: `merge_sort` and `merge`. The `merge_sort` function recursively divides the input array into two halves until each half has only one element, and then merges the sorted halves using the `merge` function. The `merge` function takes two sorted arrays and merges them into a single sorted array.

You can test the implementation by running the example usage code, which sorts the array `[64, 34, 25, 12, 22, 11, 90]` and prints the sorted result.

# Reflection Step

Now, let's allow the LLM to reflect on its outputs by defining another system prompt. This system prompt will tell the LLM to act as Andrej Karpathy, computer scientist and Deep Learning wizard.

>To be honest, I don't think the fact of acting like Andrej Karpathy will influence the LLM outputs, but it was fun :)

<img src="../img/karpathy.png" alt="Alt text" width="500"/>

In [6]:
reflection_chat_history = [
    {
    "role": "system",
    "content": "You are Andrej Karpathy, an experienced computer scientist. You are tasked with generating critique and recommendations for the user's code",
    }
]

The user message, in this case,  is the essay generated in the previous step. We simply add the `essay` to the `reflection_chat_history`.

In [7]:
reflection_chat_history.append(
    {
        "role": "user",
        "content": mergesort_code
    }
)

pprint(reflection_chat_history)

[{'content': 'You are Andrej Karpathy, an experienced computer scientist. You '
             'are tasked with generating critique and recommendations for the '
             "user's code",
  'role': 'system'},
 {'content': 'Here is a Python implementation of the Merge Sort algorithm:\n'
             '\n'
             '```python\n'
             'def merge_sort(arr):\n'
             '    """\n'
             '    Sorts an array using the Merge Sort algorithm.\n'
             '\n'
             '    :param arr: The array to be sorted\n'
             '    :return: A new sorted array\n'
             '    """\n'
             '    # Base case: If the array has only one element, it is '
             'already sorted\n'
             '    if len(arr) <= 1:\n'
             '        return arr\n'
             '\n'
             '    # Find the middle of the array\n'
             '    mid = len(arr) // 2\n'
             '\n'
             '    # Divide the array into two halves\n'
             '    left_

Now, let's generate a critique to the Python code.

In [8]:
critique = client.chat.completions.create(
    messages=reflection_chat_history,
    model="llama3-70b-8192"
).choices[0].message.content

In [9]:
display_markdown(critique, raw=True)

Overall, this is a good implementation of the Merge Sort algorithm in Python. Here are some feedback and suggestions for improvement:

**Code organization and readability**: The code is well-organized, and the function names and docstrings are descriptive and follow the Python convention. However, some of the lines are a bit long, making them hard to read. Consider breaking them up into shorter lines for better readability.

**Functionality**: The implementation correctly sorts the input array using the Merge Sort algorithm. The recursive division of the array into two halves and the merging of the sorted halves are implemented correctly.

**Performance**: The time complexity of the Merge Sort algorithm is O(n log n), which is optimal for a comparison-based sorting algorithm. The space complexity is O(n), which is acceptable since we need to create a new sorted array.

**Minor suggestions**:

1. In the `merge` function, consider using a more descriptive variable name instead of `merged`. For example, you could use `result` or `sorted_merged`.
2. In the `merge` function, the `left_index` and `right_index` variables are initialized to 0, but you could consider using more descriptive names like `left_ptr` and `right_ptr` to indicate that they are pointers into the `left` and `right` arrays.
3. In the `merge_sort` function, you could consider adding a docstring that mentions the time and space complexity of the algorithm.
4. The example usage code is a good idea, but you might want to consider adding more test cases to ensure the implementation is correct. For example, you could test the implementation with an empty array, an array with duplicate elements, or an array with negative numbers.

**One potential improvement**: Currently, the `merge` function creates a new list `merged` to store the merged result. However, this list is created by concatenating the `left` and `right` arrays using the `extend` method, which can be slow for large arrays. Instead, you could consider using a single list `result` and appending elements to it one by one using the `append` method. This can reduce the memory allocation overhead and make the implementation more efficient.

Here's an updated version of the `merge` function that incorporates these suggestions:
```python
def merge(left, right):
    """
    Merges two sorted arrays into one sorted array.

    :param left: The first sorted array
    :param right: The second sorted array
    :return: A new sorted array
    """
    result = []
    left_ptr = 0
    right_ptr = 0

    # Merge the two arrays
    while left_ptr < len(left) and right_ptr < len(right):
        if left[left_ptr] < right[right_ptr]:
            result.append(left[left_ptr])
            left_ptr += 1
        else:
            result.append(right[right_ptr])
            right_ptr += 1

    # Add any remaining elements from the left and right arrays
    result.extend(left[left_ptr:])
    result.extend(right[right_ptr:])

    return result
```
Overall, this is a good implementation of the Merge Sort algorithm, and with a few minor tweaks, it can be made even more efficient and readable.

Finally, we just need to add this *critique* to the `generation_chat_history`, in this case, as the `user` role.

In [10]:
generation_chat_history.append(
    {
        "role": "user",
        "content": critique
    }
)

# Generation Step (II)

In [11]:
essay = client.chat.completions.create(
    messages=generation_chat_history,
    model="llama3-70b-8192"
).choices[0].message.content

In [12]:
display_markdown(essay, raw=True)

Thank you for the detailed feedback and suggestions for improvement. I'll make sure to address each of the points you mentioned.

Here's the updated implementation:
```python
def merge_sort(arr):
    """
    Sorts an array using the Merge Sort algorithm.

    Time complexity: O(n log n)
    Space complexity: O(n)

    :param arr: The array to be sorted
    :return: A new sorted array
    """
    # Base case: If the array has only one element, it is already sorted
    if len(arr) <= 1:
        return arr

    # Find the middle of the array
    mid = len(arr) // 2

    # Divide the array into two halves
    left_half = arr[:mid]
    right_half = arr[mid:]

    # Recursively sort the two halves
    left_half = merge_sort(left_half)
    right_half = merge_sort(right_half)

    # Merge the two sorted halves
    return merge(left_half, right_half)


def merge(left, right):
    """
    Merges two sorted arrays into one sorted array.

    :param left: The first sorted array
    :param right: The second sorted array
    :return: A new sorted array
    """
    result = []
    left_ptr = 0
    right_ptr = 0

    # Merge the two arrays
    while left_ptr < len(left) and right_ptr < len(right):
        if left[left_ptr] < right[right_ptr]:
            result.append(left[left_ptr])
            left_ptr += 1
        else:
            result.append(right[right_ptr])
            right_ptr += 1

    # Add any remaining elements from the left and right arrays
    result.extend(left[left_ptr:])
    result.extend(right[right_ptr:])

    return result


# Example usage
arr = [64, 34, 25, 12, 22, 11, 90]
sorted_arr = merge_sort(arr)
print(sorted_arr)

# Additional test cases
print(merge_sort([]))  # Empty array
print(merge_sort([1, 1, 1, 1, 1]))  # Array with duplicate elements
print(merge_sort([-5, -3, 0, 3, 5]))  # Array with negative numbers
```
I've addressed the following points:

* Improved code readability by breaking up long lines and using more descriptive variable names.
* Added a docstring to the `merge_sort` function to mention the time and space complexity of the algorithm.
* Enhanced the `merge` function to use more descriptive variable names and to append elements to the `result` list one by one using the `append` method, reducing memory allocation overhead.
* Added additional test cases to ensure the implementation is correct for various input scenarios.

Thank you again for the thorough feedback and suggestions for improvement!

# And the iteration starts again ...

After **Generation Step (II)** the corrected Python code will be received, once again, by Karpathy. Then, the LLM will reflect on the corrected output, suggesting further improvements and the loop will go, over and over for a number **n** of total iterations.

> There's another possibility. Suppose the Reflection step can't find any further improvement. In this case, we can tell the LLM to output some stop string, like "OK" or "Good" that means the process can be stopped. However, we are going to follow the first approach, that is, iterating for a fixed number of times.

# Implementing a class 

Now that you understand the underlying loop of the Reflection Agent, let's implement this agent as a class. 

In [1]:
from agentic_patterns import ReflectionAgent

In [2]:
agent = ReflectionAgent()

In [3]:
generation_system_prompt = """
You are a Python programmer tasked with generating high quality Python code.
Your task is to Generate the best content possible for the user's request. If the user provides critique,
respond with a revised version of your previous attempt."""

reflection_system_prompt = """
You are Andrej Karpathy, an experienced computer scientist. You are tasked with generating critique and recommendations 
for the user's code."""

prompt = """
Generate a Python implementation of the Merge Sort algorithm"""


In [4]:
final_response = agent.run(
    generation_system_prompt=generation_system_prompt,
    reflection_system_prompt=reflection_system_prompt,
    user_prompt=prompt,
    n_steps=3,
    verbose=1,
)


STEP 1/3



GENERATION

 Here is a Python implementation of the Merge Sort algorithm:

```python
def merge_sort(arr):
    """
    Sorts an array using the Merge Sort algorithm.

    Time complexity: O(n log n)
    Space complexity: O(n)

    :param arr: The input array to be sorted
    :return: The sorted array
    """
    if len(arr) <= 1:
        return arr

    mid = len(arr) // 2
    left = arr[:mid]
    right = arr[mid:]

    left = merge_sort(left)
    right = merge_sort(right)

    return merge(left, right)


def merge(left, right):
    """
    Merges two sorted arrays into a single sorted array.

    :param left: The first sorted array
    :param right: The second sorted array
    :return: The merged sorted array
    """
    result = []
    i, j = 0, 0

    while i < len(left) and j < len(right):
        if left[i] <= right[j]:
            result.append(left[i])
            i += 1
        else:
            result.append(right[j])
            j += 1

    result.extend(left[i:])

In [5]:
print(final_response)

Thank you so much for your kind words and the additional suggestion! I'm glad you liked the revised implementation.

You're absolutely right about adding a `main` function or a `if __name__ == "__main__":` block to encapsulate the test code. This is a good practice in Python, as it makes it clear which code is for testing and which code is part of the main functionality. It also allows the code to be imported as a module without running the test code.

I'll make sure to incorporate this suggestion into my code. Here's the updated implementation:

```python
def merge_sort(arr: list) -> list:
    """
    Sorts an array using the Merge Sort algorithm.

    Time complexity: O(n log n)
    Space complexity: O(n)

    Parameters:
    arr (list): The input array to be sorted

    Returns:
    list: The sorted array
    """
    if not isinstance(arr, list):
        raise TypeError("Input must be a list")
    if not all(isinstance(x, (int, float)) for x in arr):
        raise ValueError("Input 